In [1]:
import laspy
import glob
from pathlib import Path
import pandas as pd
import numpy as np
import json

In [2]:
def count_hash(file):
    file = open(file)
    count = 0
    for line in file:
        if line.startswith('#'):
            count += 1
    return count

In [3]:
## PDAL method
import pdal
files = glob.glob("data/*.xyz_label_conf")
def construct_json(infile, outfile):
    skip_lines = count_hash(infile)
    pipe = {"pipeline":[]}
    pipe["pipeline"].append(
        {"type":"readers.text",
         "skip":skip_lines,
         "separator":" ",
         "header":"x y z label conf",
         "filename": infile})
    pipe["pipeline"].append(
        {"type":"writers.las",
         "filename": outfile})
    return json.dumps(pipe)

In [43]:
str(Path('data/las/').joinpath(Path(files[0]).with_suffix(".las").name))

'data/las/oakland_part3_am.las'

In [44]:
for file in files:
    outfile = str(Path('data/las/').joinpath(Path(file).with_suffix(".las").name))
    print(f'Converting {file} and saving at {outfile}')
    pipeline = pdal.Pipeline(construct_json(file, outfile))
    hold = pipeline.execute()
    print(pipeline.log)

Converting data/original/oakland_part3_am.xyz_label_conf and saving at data/las/oakland_part3_am.las

Converting data/original/oakland_part3_an.xyz_label_conf and saving at data/las/oakland_part3_an.las

Converting data/original/oakland_part2_ac.xyz_label_conf and saving at data/las/oakland_part2_ac.las

Converting data/original/oakland_part3_al.xyz_label_conf and saving at data/las/oakland_part3_al.las

Converting data/original/oakland_part3_aj.xyz_label_conf and saving at data/las/oakland_part3_aj.las

Converting data/original/oakland_part2_ag.xyz_label_conf and saving at data/las/oakland_part2_ag.las

Converting data/original/oakland_part2_ao.xyz_label_conf and saving at data/las/oakland_part2_ao.las

Converting data/original/oakland_part2_ae.xyz_label_conf and saving at data/las/oakland_part2_ae.las

Converting data/original/oakland_part2_ak.xyz_label_conf and saving at data/las/oakland_part2_ak.las

Converting data/original/oakland_part2_ah.xyz_label_conf and saving at data/las/oa

In [29]:
## First way I did this...PDAL method is better.
files = glob.glob("data/original/*.xyz_label_conf")
bad_files = []
for file in files:
    try:
        print(file)
        new_file = Path(file.split('.')[0]+'.las')
        outfile_temp = Path.cwd().joinpath("data/las/" + str(new_file.name))
        outfile_hdr = laspy.header.Header()
        outfile = laspy.file.File(outfile_temp, mode='w', header=outfile_hdr)
        df = pd.read_csv(file, 
                     names=['x','y','z','label','conf'],
                     delimiter = ' ',
                     skiprows=63)
        outfile.header.max = [max(df['x']), max(df['y']), max(df['z'])]
        minx = min(df['x'])
        miny = min(df['y'])
        minz = min(df['z'])
        outfile.header.min = [minx, miny, minz]
        outfile.header.offset = [0., 0., 0.]
        outfile.header.scale = [.01, .01, .01] 
        df['x'] = df['x']*100
        df['y'] = df['y']*100
        df['z'] = df['z']*100
        df = df.astype('int64')
        outfile.return_num = np.ones(len(df['x'])).astype(np.uint8)
        outfile.X = df['x']
        outfile.Y = df['y']
        outfile.Z = df['z']
        #outfile.return_num = np.ones(len(df['x']))
        #outfile.num_returns = np.ones(len(df['x']))
        outfile.close()
    except:
        bad_files.append(file)

data/original/oakland_part3_am.xyz_label_conf
data/original/oakland_part3_an.xyz_label_conf
data/original/oakland_part2_ac.xyz_label_conf
data/original/oakland_part3_al.xyz_label_conf
data/original/oakland_part3_aj.xyz_label_conf
data/original/oakland_part2_ag.xyz_label_conf
data/original/oakland_part2_ao.xyz_label_conf
data/original/oakland_part2_ae.xyz_label_conf
data/original/oakland_part2_ak.xyz_label_conf
data/original/oakland_part2_ah.xyz_label_conf
data/original/oakland_part2_ai.xyz_label_conf
data/original/oakland_part3_ao.xyz_label_conf
data/original/oakland_part2_ai_new.xyz_label_conf
data/original/oakland_part2_ad.xyz_label_conf
data/original/oakland_part3_ap.xyz_label_conf
data/original/oakland_part3_ak.xyz_label_conf


In [12]:
bad_files

['data/original/oakland_part3_al.xyz_label_conf',
 'data/original/oakland_part3_aj.xyz_label_conf']